In [ ]:
import json
import os
import random

from templates.filter import *
from templates.func import *
from templates.QA import QADataset


In [ ]:
DATASET = "scannetpp"
os.chdir(f"/mnt/bn/nlhei-nas/liubangya/proj/vlm/datasets/{DATASET}")

DS_ROOT = f"/mnt/bn/nlhei-nas/liubangya/proj/vlm/datasets/{DATASET}/structured-data"
CAP_FILE = f"/mnt/bn/nlhei-nas/liubangya/proj/vlm/datasets/{DATASET}/captions_all.yaml"

N_TRAIN = 100_000
N_TEST = 2_000

def scene_valid_fn(sc_name):
    scene_path = os.path.join(DS_ROOT, sc_name)
    finished_flag = "finished.flag"
    finished_flag_path = os.path.join(scene_path, finished_flag)
    return os.path.exists(finished_flag_path)

random.seed(0)
myCap = Captioner(CAP_FILE) if os.path.exists(CAP_FILE) else None
ds = QADataset(DS_ROOT, myCap)

In [ ]:
# img/video related arguments
grid_cfg_file = f"/mnt/bn/nlhei-nas/liubangya/proj/vlm/QA/grid_cfg_{DATASET}.json"
with open(grid_cfg_file, "r") as f:
    grid_cfg = json.load(f)
H = grid_cfg["H"]
W = grid_cfg["W"]
patchsize_H = grid_cfg["patchsize_H"]
patchsize_W = grid_cfg["patchsize_W"]
n_frames = 6
frame_stride = 4


tasks_cfg = {
    "total_QAs": 100000,
    "roi_frame_only": True,
    "H": H, "W": W, "patchsize_H": patchsize_H, "patchsize_W": patchsize_W,
    "motion_thres": 4, "num_frame": n_frames, "frame_stride": frame_stride,
    "prefix": "scannetpp_new"
}


total_qas = tasks_cfg["total_QAs"]
H_grids = tasks_cfg["H"] // tasks_cfg["patchsize_H"]
W_grids = tasks_cfg["W"] // tasks_cfg["patchsize_W"]
prefix = tasks_cfg["prefix"]

task_dir = f"/mnt/bn/nlhei-nas/liubangya/proj/vlm/workspace/image_{prefix}_{H_grids}x{W_grids}_{n_frames}"
OUTPUT_DIR = os.path.join(task_dir, "pairs")
OUTPUT_QWEN = f"{OUTPUT_DIR}/QA_pairs_qwen.json"
OUTPUT_JSON = f"{OUTPUT_DIR}/QA_pairs.json"
os.makedirs(OUTPUT_DIR, exist_ok=True)


task_obj = {
    os.path.basename(task_dir): {
        "train_qa": OUTPUT_QWEN.replace(".json", ".train.json"),
        "test_qa": OUTPUT_QWEN.replace(".json", ".test.json"),
        "train_qa_meta": OUTPUT_JSON.replace(".json", ".train.json"),
        "test_qa_meta": OUTPUT_JSON.replace(".json", ".test.json"),
    }
}
print(json.dumps(task_obj, indent=4))